In [1]:
#importing all the libraries 
import numpy as np 
import pandas as pd 
import os 
import joblib as jb
from collections import defaultdict
from itertools import groupby
from math import sqrt, atan2
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from tensorflow.keras.layers import Input, Flatten, Dropout, Dense, Concatenate
from tensorflow.keras.models import Model
from tcn import TCN
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss


2024-08-05 07:33:24.611472: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 07:33:24.611509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 07:33:24.612802: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 07:33:24.619056: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 07:33:25.290007: W tensorflow/compiler/tf2

In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pratham21082 (pratham_03). Use `wandb login --relogin` to force relogin


True

In [3]:
wandb.init(project = "ISAC_2_TCN",name="Phase1_ph_new (5 epochs)")

In [4]:
path = '/home/vijay/paper_jc/conference/phase1_ph_student_label'
all_files=[]
for subfolder in os.listdir(path):
        subfolder_path = os.path.join(path, subfolder)
        if os.path.isdir(subfolder_path):
            files = os.listdir(subfolder_path)
            files.sort()
            for file in files:
                file_path = os.path.join(subfolder_path, file)
                df = pd.read_csv(file_path, index_col=0)
                all_files.append(df)

    

In [5]:
df = pd.concat(all_files, axis=0 )
df  = df.reset_index(drop=True)
print(df.isnull().values.any())
Y = df['headlabel']
X = df.iloc[0:,0:-1]
selector = SelectKBest(f_classif, k=25)
csi_feature_25 = selector.fit_transform(X, Y)

False


In [6]:
csi_feature_5X5=np.reshape(csi_feature_25,(csi_feature_25.shape[0],5,5))
csi_feature_5X5.shape

(1970832, 5, 5)

In [7]:
Y = pd.DataFrame(Y)
onehot_encoder = OneHotEncoder()
csi_headlabel= onehot_encoder.fit_transform(Y)

In [8]:

X_train, X_ , y_train,y_ = train_test_split(csi_feature_5X5, csi_headlabel, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val=train_test_split(X_, y_, test_size=0.5, random_state=42)

In [9]:
y_test = y_test.toarray()
y_val = y_val.toarray()
y_train = y_train.toarray()

In [10]:
def csi_network_inc_res(input_sh, output_sh):
    nb_filters = 64
    x_input = Input(input_sh)
    tcn1 = TCN(
        nb_filters=nb_filters,
        nb_stacks = 1,
        kernel_size=10,
        dilations=(1, 2, 4, 8, 16),
        use_layer_norm=True,
    )
    tcn2 = TCN(
        nb_filters=nb_filters,
        nb_stacks = 1,
        kernel_size=10,
        dilations=(1,2, 4, 8, 16,32),
        use_layer_norm=True,
    )
    tcn3 = TCN(
        nb_filters=nb_filters,
        nb_stacks = 1,
        kernel_size=10,
        dilations=(2, 4, 8, 16,32,64),
        use_layer_norm=True,
    )
    x = tcn1(x_input)
    print(f'TCN.receptive_field: {tcn1.receptive_field}.')
    x = tcn2(x_input)
    print(f'TCN.receptive_field: {tcn2.receptive_field}.')
    x = tcn3(x_input)
    print(f'TCN.receptive_field: {tcn3.receptive_field}.')
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(output_sh, activation='sigmoid', name='dense')(x)
    
    model = Model(inputs=x_input, outputs=x, name='csi_model')
    return model

In [11]:
model=csi_network_inc_res((5, 5), 7)
def log_metrics(epoch, logs):

    wandb.log(logs, step=epoch)

wandb_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_metrics)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit( X_train,y_train, batch_size=100,epochs=5,validation_data=[X_val, y_val], callbacks=[
        tf.keras.callbacks.ModelCheckpoint('/home/vijay/paper_jc/conference/models/ph1_tcn.hdf5',
                                            monitor='val_accuracy',
                                            save_best_only=True,
                                            save_weights_only=False),
        wandb_callback
    ])

2024-08-05 07:34:31.267317: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-05 07:34:32.366220: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-05 07:34:32.366497: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

TCN.receptive_field: 559.
TCN.receptive_field: 1135.
TCN.receptive_field: 2269.
Epoch 1/5


2024-08-05 07:34:40.097075: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-08-05 07:34:41.747301: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-05 07:34:43.618051: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-05 07:34:44.914820: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f6600a12500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-05 07:34:44.914865: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA L40, Compute Capability 8.9
2024-08-05 07:34:45.013163: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1722843285.251599  844632 device_compiler.h:186] Compil

11825/11825 [==============================] - 256s 21ms/step - loss: 0.1640 - accuracy: 0.9514 - val_loss: 0.0380 - val_accuracy: 0.9889
Epoch 2/5
    1/11825 [..............................] - ETA: 3:56 - loss: 0.0097 - accuracy: 1.0000

/home/vijay/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11825/11825 [==============================] - 257s 22ms/step - loss: 0.0379 - accuracy: 0.9898 - val_loss: 0.0430 - val_accuracy: 0.9866
Epoch 3/5
11825/11825 [==============================] - 263s 22ms/step - loss: 0.0232 - accuracy: 0.9939 - val_loss: 0.0249 - val_accuracy: 0.9940
Epoch 4/5
11825/11825 [==============================] - 261s 22ms/step - loss: 0.0168 - accuracy: 0.9955 - val_loss: 0.0111 - val_accuracy: 0.9969
Epoch 5/5
11825/11825 [==============================] - 259s 22ms/step - loss: 0.0131 - accuracy: 0.9967 - val_loss: 0.0091 - val_accuracy: 0.9978


In [13]:
model.save('finaltcnmodel.h5')

In [14]:
wandb.finish()

accuracy,▁▇███
loss,█▂▁▁▁
val_accuracy,▂▁▆▇█
val_loss,▇█▄▁▁
accuracy,0.99666
loss,0.01308
val_accuracy,0.99784
val_loss,0.00911


## Phase 2 - FSL

In [15]:
wandb.init(project = "ISAC_2",name="Phase2_ph_new (No Retraining on Phase 2) (Only Trained on Phase 1)")

In [16]:
path_test = '/home/vijay/paper_jc/conference/phase2_ph_new'
files_test= os.listdir(path_test)
files_test.sort()
all_files_test=[]
for i in files_test:
    df_test = pd.read_csv(path_test + '/' + i, index_col=0)
    all_files_test.append(df_test)

df_test = pd.concat(all_files_test, axis=0 )
df_test  = df_test.reset_index(drop=True)
Y_test_ = df_test['headlabel']
X_test_ = df_test.iloc[0:,0:-1]
selector = SelectKBest(f_classif, k=25)
csi_feature_25_test = selector.fit_transform(X_test_, Y_test_)

In [21]:
Y_test_ = pd.DataFrame(Y_test_)
onehot_encoder = OneHotEncoder(sparse=False)
csi_headlabel_test= onehot_encoder.fit_transform(Y_test_)
csi_feature_5X5_test=np.reshape(csi_feature_25_test,(csi_feature_25_test.shape[0],5,5))
print(csi_feature_5X5_test.shape)
X_testph2, X__ , y_testph2,y__ = train_test_split(csi_feature_5X5_test, csi_headlabel_test, test_size=0.01, random_state=42)

/home/vijay/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(608094, 5, 5)


In [22]:
train, val, train_label, val_label = train_test_split(X__, y__, test_size=0.5, random_state=42)

In [23]:
path_to_model = '/home/vijay/paper_jc/varun/finaltcnmodel.h5'
model = tf.keras.models.load_model(path_to_model, custom_objects={'TCN': TCN})

In [27]:
def log_metrics(epoch, logs):
    wandb.log(logs, step=epoch)

wandb_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_metrics)

model.fit(
    train,
    train_label,
    batch_size=64,
    epochs=10,
    validation_data=[val, val_label],
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('/home/vijay/journal_paper/model_p1_p2_ph_new_15epochs_1%.hdf5',
                                            monitor='val_accuracy',
                                            save_best_only=True,
                                            save_weights_only=False),
    ]
)

Epoch 1/10
48/48 [==============================] - 1s 31ms/step - loss: 0.2086 - accuracy: 0.9438 - val_loss: 0.3572 - val_accuracy: 0.9089
Epoch 2/10
 1/48 [..............................] - ETA: 1s - loss: 0.0825 - accuracy: 0.9844

/home/vijay/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


48/48 [==============================] - 1s 29ms/step - loss: 0.1230 - accuracy: 0.9681 - val_loss: 0.3840 - val_accuracy: 0.9102
Epoch 3/10
48/48 [==============================] - 1s 28ms/step - loss: 0.0870 - accuracy: 0.9760 - val_loss: 0.3804 - val_accuracy: 0.9165
Epoch 4/10
48/48 [==============================] - 2s 32ms/step - loss: 0.0871 - accuracy: 0.9737 - val_loss: 0.3647 - val_accuracy: 0.9188
Epoch 5/10
48/48 [==============================] - 1s 31ms/step - loss: 0.0448 - accuracy: 0.9875 - val_loss: 0.3780 - val_accuracy: 0.9270
Epoch 6/10
48/48 [==============================] - 1s 30ms/step - loss: 0.0260 - accuracy: 0.9931 - val_loss: 0.3917 - val_accuracy: 0.9303
Epoch 7/10
48/48 [==============================] - 1s 25ms/step - loss: 0.0201 - accuracy: 0.9954 - val_loss: 0.4596 - val_accuracy: 0.9234
Epoch 8/10
48/48 [==============================] - 1s 26ms/step - loss: 0.0303 - accuracy: 0.9924 - val_loss: 0.4429 - val_accuracy: 0.9227
Epoch 9/10
48/48 [======

In [28]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
y_pred = model.predict(X_testph2)
y_predicted = np.argmax(y_pred, axis=1)
ground_truth = np.argmax(y_testph2, axis=1)

accuracy_test = accuracy_score(ground_truth, y_predicted)

conf_matrix = confusion_matrix(ground_truth, y_predicted)

f1 = f1_score(ground_truth, y_predicted, average='micro')

print("Accuracy_Test:", accuracy_test)
print("Confusion Matrix:\n", conf_matrix)
print("F1 Score:", f1)

headlabel_accuracy = {"Forward": 0.0, "Looking Down": 0.0, "Looking Up": 0.0, "Looking Left": 0.0, "Looking Right": 0.0, "Nodding": 0.0, "Shaking": 0.0}

for i, headlabel in enumerate(headlabel_accuracy.keys()):
    accuracy = conf_matrix[i, i] / conf_matrix[i].sum() if conf_matrix[i].sum() > 0 else 0.0
    headlabel_accuracy[headlabel] = accuracy

print(headlabel_accuracy)


18813/18813 [==============================] - 140s 7ms/step
Accuracy_Test: 0.9186030866443083
Confusion Matrix:
 [[123271   1289    321   1575    690    829   1060]
 [  1923  93393   1327   1100   1083   3921   1656]
 [  1067    626  39990    806    679   1569    920]
 [  1661    890    102  61954   1114   2766   1894]
 [  1341    581    987   1580  37065   1194   2565]
 [  1036   1039    513    475    340 138503    837]
 [   859    640    559   1382    575   1631  58835]]
F1 Score: 0.9186030866443082
{'Forward': 0.9553299492385787, 'Looking Down': 0.8945432602511422, 'Looking Up': 0.8758788356659439, 'Looking Left': 0.8802659808755204, 'Looking Right': 0.8179771809414517, 'Nodding': 0.970296266717107, 'Shaking': 0.9124393232114887}


## Phase 3 - FSL

In [29]:
wandb.init(project = "ISAC_2",name="Phase 3 1% 15 epochs (Phase 2 1%)")

In [30]:
path_test = '/home/vijay/paper_jc/conference/phase3_ph_new'
files_test= os.listdir(path_test)
files_test.sort()
all_files_test=[]
for i in files_test:
    df_test = pd.read_csv(path_test + '/' + i, index_col=0)
    all_files_test.append(df_test)

df_test = pd.concat(all_files_test, axis=0 )
df_test  = df_test.reset_index(drop=True)
Y_test_ = df_test['headlabel']
X_test_ = df_test.iloc[0:,0:-1]
selector = SelectKBest(f_classif, k=25)
csi_feature_25_test = selector.fit_transform(X_test_, Y_test_)

In [39]:
Y_test_ = pd.DataFrame(Y_test_)
onehot_encoder = OneHotEncoder(sparse=False)
csi_headlabel_test= onehot_encoder.fit_transform(Y_test_)
csi_feature_5X5_test=np.reshape(csi_feature_25_test,(csi_feature_25_test.shape[0],5,5))
csi_feature_5X5_test.shape
X_testph3, X__ , y_testph3,y__ = train_test_split(csi_feature_5X5_test, csi_headlabel_test, test_size=0.01, random_state=42)
train, val, train_label, val_label = train_test_split(X__, y__, test_size=0.5, random_state=42)

/home/vijay/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [40]:
path_to_model = '/home/vijay/paper_jc/varun/finaltcnmodel.h5'
model = tf.keras.models.load_model(path_to_model, custom_objects={'TCN': TCN})

In [41]:
def log_metrics(epoch, logs):
    wandb.log(logs, step=epoch)

wandb_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_metrics)

model.fit(
    train,
    train_label,
    batch_size=64,
    epochs=17,
    validation_data=[val, val_label],
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('/home/vijay/journal_paper/model_p1_p2_p3_17epochs_1%.hdf5',
                                            monitor='val_accuracy',
                                            save_best_only=True,
                                            save_weights_only=False),
        wandb_callback
    ]
)

Epoch 1/17
76/76 [==============================] - 8s 45ms/step - loss: 1.9588 - accuracy: 0.3720 - val_loss: 1.3241 - val_accuracy: 0.5119
Epoch 2/17
 1/76 [..............................] - ETA: 1s - loss: 1.3568 - accuracy: 0.5000

/home/vijay/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


49/76 [==================>...........] - ETA: 0s - loss: 1.2813 - accuracy: 0.5424

wandb: WARNING (User provided step: 0 is less than current step: 16. Dropping entry: {'loss': 1.9588371515274048, 'accuracy': 0.3720157742500305, 'val_loss': 1.3240524530410767, 'val_accuracy': 0.5119369029998779, '_timestamp': 1722846404.0392122}).


76/76 [==============================] - 2s 27ms/step - loss: 1.2414 - accuracy: 0.5524 - val_loss: 1.0422 - val_accuracy: 0.6355
Epoch 3/17
45/76 [================>.............] - ETA: 0s - loss: 1.0134 - accuracy: 0.6476

...

76/76 [==============================] - 2s 26ms/step - loss: 0.1075 - accuracy: 0.9676 - val_loss: 0.3106 - val_accuracy: 0.9288
Epoch 16/17
49/76 [==================>...........] - ETA: 0s - loss: 0.0883 - accuracy: 0.9770

wandb: WARNING (User provided step: 14 is less than current step: 16. Dropping entry: {'loss': 0.1075383722782135, 'accuracy': 0.9676147103309631, 'val_loss': 0.3106214106082916, 'val_accuracy': 0.9287938475608826, '_timestamp': 1722846433.9602811}).


76/76 [==============================] - 2s 27ms/step - loss: 0.0915 - accuracy: 0.9759 - val_loss: 0.3065 - val_accuracy: 0.9323
Epoch 17/17
47/76 [=================>............] - ETA: 0s - loss: 0.0728 - accuracy: 0.9791

wandb: WARNING (User provided step: 15 is less than current step: 16. Dropping entry: {'loss': 0.09151672571897507, 'accuracy': 0.9759186506271362, 'val_loss': 0.3065442740917206, 'val_accuracy': 0.9323230385780334, '_timestamp': 1722846436.0134344}).


76/76 [==============================] - 2s 29ms/step - loss: 0.0655 - accuracy: 0.9807 - val_loss: 0.3048 - val_accuracy: 0.9363


In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
y_pred = model.predict(X_testph3)
y_predicted = np.argmax(y_pred, axis=1)
ground_truth = np.argmax(y_testph3, axis=1)

accuracy_test = accuracy_score(ground_truth, y_predicted)

conf_matrix = confusion_matrix(ground_truth, y_predicted)

f1 = f1_score(ground_truth, y_predicted, average='micro')

print("Accuracy_Test:", accuracy_test)
print("Confusion Matrix:\n", conf_matrix)
print("F1 Score:", f1)

headlabel_accuracy = {"Forward": 0.0, "Looking Down": 0.0, "Looking Up": 0.0, "Looking Left": 0.0, "Looking Right": 0.0, "Nodding": 0.0, "Shaking": 0.0}

for i, headlabel in enumerate(headlabel_accuracy.keys()):
    accuracy = conf_matrix[i, i] / conf_matrix[i].sum() if conf_matrix[i].sum() > 0 else 0.0
    headlabel_accuracy[headlabel] = accuracy

print(headlabel_accuracy)

29805/29805 [==============================] - 224s 7ms/step
Accuracy_Test: 0.9367621216505827
Confusion Matrix:
 [[232102   1425    205    270    305   1932   1035]
 [  3161 116297   1396    940   1639   1801    863]
 [   939   1630  57194    971   2693    271    570]
 [  1892   2103   3791  90618   1847   2989   2137]
 [  1672   1078   2309   1437  55318   1763   1003]
 [  3970    406    257   1053    574 224331    699]
 [  3322    840    915    762    622    826 117575]]
F1 Score: 0.9367621216505827
{'Forward': 0.9782024157724825, 'Looking Down': 0.9222820527054569, 'Looking Up': 0.8899296695089314, 'Looking Left': 0.8599409738367955, 'Looking Right': 0.8565809848250232, 'Nodding': 0.9699122313978122, 'Shaking': 0.9416395700853742}
